In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import Imputer
import gc
import numpy as np
#from joblib import dump, load
from sklearn import metrics
#import warnings
#warnings.filterwarnings("ignore")
from datetime import datetime
from joblib import dump, load

In [2]:
#train = pd.read_csv('./data/processed/train_processed.csv',index_col='TransactionID')#, nrows=20000)
train = pd.read_pickle("./data/processed/train_processed.pkl")
train.shape
feature_names = list(train.columns)
feature_names.pop(0)

'isFraud'

**Split into test and train:**

In [3]:
gc.collect()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.loc[:, train.columns != 'isFraud'], train['isFraud'], test_size=0.2)
del train
gc.collect()

17

**Base model**:

In [8]:
start=datetime.now()
print(start)
gc.collect()
base_model = RandomForestClassifier(random_state=42,n_estimators = 850, max_depth=25)
print(base_model.get_params())
base_model.fit(X_train, y_train)
finish=datetime.now()
print('time taken: '+str(finish-start))
important_features = pd.DataFrame(base_model.feature_importances_,feature_names).sort_values(0,ascending=False).head(10)

{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 25, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 850, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
time taken: 4 days, 18:10:41.791198


test model on test data

In [18]:
print(datetime.now())
y_pred_prob=base_model.predict_proba(X_test)
y_pred=base_model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("ROC area under:",metrics.roc_auc_score(y_test, y_pred_prob[:,1]))
print(datetime.now())

2019-09-05 13:54:52.344844
Accuracy: 0.9766654248653774
ROC area under: 0.9174208715163343
2019-09-05 13:54:56.679103


In [63]:
datetimestring=datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
dump(base_model, './data/interim/'+datetimestring+'_random_foest_base.joblib')

['./data/interim/04-09-2019_14-45-04_random_foest_base.joblib']

In [4]:
import warnings
warnings.filterwarnings("ignore")
start=datetime.now()
grid_results=pd.read_pickle("./data/interim/grid_search_results.pkl")
rf = RandomForestClassifier(random_state = 42)
param = {'n_estimators': [150,250],
        'max_depth': [25]}
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1,scoring='roc_auc',verbose=2)

**Grid search with CV**

In [7]:
import warnings
warnings.filterwarnings("ignore")
start=datetime.now()
grid_results=pd.read_pickle("./data/interim/grid_search_results.pkl")
rf = RandomForestClassifier(random_state = 42)
param = {'n_estimators': [150],
        'max_depth': [25]}
gs = GridSearchCV(rf, param, cv=3, n_jobs=-1,scoring='roc_auc',verbose=2)
gs_fit = gs.fit(X_train, y_train)
grid_results = pd.concat([grid_results,pd.DataFrame(gs_fit.cv_results_)]).sort_values('mean_test_score', ascending=False)
#grid_results = pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)
grid_results.to_pickle("./data/interim/grid_search_results.pkl")
finish=datetime.now()
print('time taken: '+str(finish-start))
grid_results[['params','mean_fit_time','mean_test_score']]

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 12.8min finished


KeyboardInterrupt: 

In [6]:
grid_results[['params','mean_fit_time','mean_test_score']]

,params,mean_fit_time,mean_test_score
1,"{'max_depth': 25, 'n_estimators': 75}",518.037236,0.913438
0,"{'max_depth': 25, 'n_estimators': 50}",503.805331,0.910337
1,"{'max_depth': 25, 'n_estimators': 20}",191.217247,0.898824
0,"{'max_depth': 25, 'n_estimators': 10}",157.243171,0.881847
1,"{'max_depth': 50, 'n_estimators': 20}",212.623711,0.876280
1,"{'max_depth': 10, 'n_estimators': 20}",80.735949,0.864587
0,"{'max_depth': 10, 'n_estimators': 10}",76.021861,0.860753
0,"{'max_depth': 50, 'n_estimators': 10}",183.580265,0.847474


**Random Grid Search:**

In [ ]:
print(datetime.now())
gc.collect()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
#n_estimators = [int(x) for x in np.linspace(start = 10, stop = 20, num = 2)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
               #'bootstrap': bootstrap}
              }
print(random_grid)
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
print(rf_random.best_params_)
print(datetime.now())

{'n_estimators': [200, 288, 377, 466, 555, 644, 733, 822, 911, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]}
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


**create model from prams found in random search **

NameError: name 'datetime' is not defined

**Get previously saved grid results:**

In [71]:
grid_results=pd.read_pickle("./grid_search_results.pkl")
grid_results[['params','mean_fit_time','mean_test_score']]

,params,mean_fit_time,mean_test_score
2,"{'max_depth': 25, 'n_estimators': 1000}",201.085752,0.869737
1,"{'max_depth': 25, 'n_estimators': 750}",153.248274,0.869110
0,"{'max_depth': 25, 'n_estimators': 500}",85.772179,0.868095
7,"{'max_depth': 100, 'n_estimators': 750}",167.374139,0.867422
10,"{'max_depth': 250, 'n_estimators': 750}",156.919886,0.867422
4,"{'max_depth': 50, 'n_estimators': 750}",152.213682,0.867311
8,"{'max_depth': 100, 'n_estimators': 1000}",217.003304,0.867248
11,"{'max_depth': 250, 'n_estimators': 1000}",189.435266,0.867248
5,"{'max_depth': 50, 'n_estimators': 1000}",208.036776,0.867164
6,"{'max_depth': 100, 'n_estimators': 500}",101.418571,0.866536


**Do a grid search around this area, load from old results and add to it:**

In [73]:
#grid_results=pd.read_pickle("./grid_search_results.pkl")
rf = RandomForestClassifier(random_state = 42)
param = {'n_estimators': [1250,1500],
        'max_depth': [25,50]}

gs = GridSearchCV(rf, param, cv=10, n_jobs=-1,scoring='roc_auc')
gs_fit = gs.fit(X_train, y_train)
grid_results = pd.concat([grid_results,pd.DataFrame(gs_fit.cv_results_)]).sort_values('mean_test_score', ascending=False)
#grid_results = pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)
grid_results.to_pickle("./grid_search_results.pkl")
grid_results[['params','mean_fit_time','mean_test_score']]

,params,mean_fit_time,mean_test_score
0,"{'max_depth': 25, 'n_estimators': 1250}",244.970298,0.870784
1,"{'max_depth': 25, 'n_estimators': 1500}",300.694907,0.869866
2,"{'max_depth': 25, 'n_estimators': 1000}",201.085752,0.869737
1,"{'max_depth': 25, 'n_estimators': 750}",153.248274,0.869110
0,"{'max_depth': 25, 'n_estimators': 500}",85.772179,0.868095
7,"{'max_depth': 100, 'n_estimators': 750}",167.374139,0.867422
10,"{'max_depth': 250, 'n_estimators': 750}",156.919886,0.867422
4,"{'max_depth': 50, 'n_estimators': 750}",152.213682,0.867311
8,"{'max_depth': 100, 'n_estimators': 1000}",217.003304,0.867248
11,"{'max_depth': 250, 'n_estimators': 1000}",189.435266,0.867248


In [74]:
model = RandomForestClassifier(n_estimators = 1250, random_state = 42, max_depth=25)
model.fit(X_train, y_train)
y_pred_prob=model.predict_proba(X_test)
y_pred=model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("ROC area under:",metrics.roc_auc_score(y_test, y_pred_prob[:,1]))

Accuracy: 0.97875
ROC area under: 0.8700979240109674


In [76]:
dump(model, 'randomforest_model_new.joblib')

['randomforest_model_new.joblib']

In [51]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

                         train: 815.5MiB
                       X_train: 651.9MiB
                        X_test: 163.0MiB
                       y_train:   4.1MiB
                        y_test:   1.0MiB
                            __:  92.0KiB
                           ___:  92.0KiB
                           _44:  92.0KiB
                           _46:  92.0KiB
                           _48:  92.0KiB
